**NOTE** `phimats` environment should be used as kernel

In [2]:
import numpy as np
import h5py

from PreProcessing import PhysicsConfig, MeshConfig, PreProcessing as PP

from MeshManager import MeshManager

from BoundaryConditions import *

from PostProcessing import WriteXDMF

%load_ext autoreload
%autoreload 2

### Simulation data

In [19]:
SimulName = "FluxGB"
# Element sets
nElementSets = 1
# Simulation type
SimulType = "GBTrapping" 
# Number of steps to achieve the load
nSteps = 100

### Read mesh file 

The mesh file is hosted on Zenodo due to its large size.  
To download it, paste the command below into your terminal.  
**Note:** Make sure you're in the desired working directory.

```bash
wget https://zenodo.org/records/18369054/files/RVE_mesh.msh
```

In [20]:
!wget https://zenodo.org/records/18369054/files/RVE_mesh.msh

--2026-01-25 19:31:58--  https://zenodo.org/records/18369054/files/RVE_mesh.msh
Resolving zenodo.org (zenodo.org)... 188.185.43.153, 137.138.52.235, 188.185.48.75, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114754226 (109M) [application/octet-stream]
Saving to: ‘RVE_mesh.msh.1’

RVE_mesh.msh.1       33%[=====>              ]  36.47M  1.35MB/s    eta 49s    

In [21]:
# Element name
elementName = "triangle"  		# meshio compatible element name
mesh = MeshManager("RVE_mesh.msh", elementName)
mesh.WriteMesh(SimulName)

Reading mesh: RVE_mesh.msh...

Mesh consistency verified: All domain elements are <triangle>.

PHIMATS MESH INITIATION
Detected 1 Material Sets. Follow this order in your Input Data:
  [Index 1] -> Physical Group: 'RVE'

------------------------------
MESH SUMMARY
------------------------------
File Name:      RVE_mesh.msh
Element Type:   triangle
Dimension:      2D
Node Order:     1
Total Nodes:    1000000
Total Elements: 1996002
Cell Blocks:
  - line: 3996 items
  - triangle: 1996002 items
------------------------------



In [22]:
# Create the config object
meshConfig = MeshConfig(
    nTotNodes=mesh.get_nTotNodes(),
    nTotElements=mesh.get_nTotElements(),
    nDim=mesh.get_nDim(),
    materialNames=mesh.getMaterialNames(),
)

### Read phase-field RVE data

In [23]:
# gPhi mapped to finite element mesh

fh5 = h5py.File("GB.rve.hdf5", "r")

try:
	gPhi = fh5["gPhi"][()]    
	fh5.close()
except:
    fh5.close()


## Diffusion and trapping data

In [24]:
# Diffusivity data
T = 300                    # Temperature [K]
R = 8.31446261815324       # Universal gas constant [J/mol.K]
dt = 10e-3  			   # Time increment [s]
N = 6          	# Number of interstitial sites per reference lattice atom [tetrahedral sites in BCC]

D01 = 8.45e-8; DQ1 = 5000  # Reference lattice diffusivity m²/s
DL = D01*np.exp(-DQ1/(R*T))

D02 = 8.45e-8; DQ2 = 5000  # GB diffusivity m²/s
DT = D02*np.exp(-DQ2/(R*T))

HGB = 10000 # GB enrichment enthalpy[J/mol]

Vm = 7.09e-6  # Molar volume of Fe [m³/mol]
Vgb = 2*Vm    # Molar volume around GBs  [m³/mol]

kappa_GB = 4*N*HGB/Vm # GB occupancy enrichment ratio
KGB = np.exp(0.25*kappa_GB*Vm/(N*R*T))
print("KGB: ", KGB)

theta_b = 3.453e-3*Vm/N
print(theta_b)

cL = theta_b*N/Vm  # Concentration  [mol/m³]
print("cL", cL)

theta_gb = theta_b*KGB # Occupancy at the grain boundaries

c_GB = theta_gb/Vgb
print("c_GB", c_GB)
Z_gb = c_GB/cL # Concentration enrichment ratio
print("Z_gb", Z_gb)
zeta_gb = 4*R*T*np.log(Z_gb)
print("zeta_gb", zeta_gb)

print("Eq Con GB: ", np.exp(0.25*zeta_gb/(R*T)))
print("Eq Con TJ: ", np.exp(0.33*zeta_gb/(R*T)))

KGB:  55.09607620783739
4.080295e-09
cL 0.003453
c_GB 0.01585389592880521
Z_gb 4.591339683986449
zeta_gb 15207.20386088472
Eq Con GB:  4.591339683986449
Eq Con TJ:  7.477562307558887


In [25]:
# Material parameters dict

Materials = {
    "RVE" : {
	"D0x1" : D01,
 	"D0y1" : D01,
	"DQx1" : DQ1,
	"DQy1" : DQ1,
 	"D0x2" : D02,
 	"D0y2" : D02,
	"DQx2" : DQ2,
	"DQy2" : DQ2,
 	"zeta_GB"  : zeta_gb,
	}
}

Materials

{'RVE': {'D0x1': 8.45e-08,
  'D0y1': 8.45e-08,
  'DQx1': 5000,
  'DQy1': 5000,
  'D0x2': 8.45e-08,
  'D0y2': 8.45e-08,
  'DQx2': 5000,
  'DQy2': 5000,
  'zeta_GB': np.float64(15207.20386088472)}}

### Apply boundary conditions

In [ ]:
# Dirichlet BCs list

exitNods = mesh.getNodesByGroup("right")
conBCs = []

# Entry con
conBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("left"), dof=0, value=cL))

# Exit con
conBCs.extend(AssignDirichletBC(mesh.getNodesByGroup("right"), dof=0, value=0))

# Equilibrium boundary conditions
for i in range(len(conBCs)):
    conBCs[i][2] = conBCs[i][2]*np.exp(gPhi[conBCs[i][0]]*zeta_gb/(R*T))

# Write external 
WriteBCVTK(SimulName, mesh, conBCs, dofNames=['con'])

BC visualization written to FluxGB_BC.vtu


In [27]:
conBCs = [[row[0], row[-1]] for row in conBCs]

### Initialize simulation object

In [28]:
# Create the config object
diffPhysConfig = PhysicsConfig(
    SimulName = SimulName,
    PhysicsType = "Transport",
    PhysicsCategory = SimulType,
    nSteps    = nSteps,
    presBCs=conBCs,
    exitNodes = exitNods,
    dt = dt,
    Temperature = T,
)

In [29]:
DiffGB = PP(diffPhysConfig, meshConfig, Materials)


PHIMATS: FluxGB | Transport (GBTrapping)
  Nodes/Elems : 1000000 / 1996002
  Total DOFs  : 1000000
  BCs Count   : 2000



In [30]:
DiffGB.WriteInputFile()

  Input file initialized: FluxGB.diff.in.hdf5


In [31]:
DiffGB.WriteOutputFile(OVERWRITE=True, AVCON=True, FLUX=True, AVFLUX=True)

  Output file initialized: FluxGB.diff.out.hdf5


### Generate xdmf visualization file

In [32]:
WriteXDMF(SimulName, # Base simulation name
          elementName, # Element name
          nSteps+1, # Number of steps 
          components=["diff"],
		  nDim=2,
		  # mechModel="Plastic", # Adds strain_p, stress_eq, etc.
		  FLUX=True)